### **Extract results for Climate Explorer spreadsheet from validation / attribution output**


In [1]:
import sys; sys.path.append('/home/clair/WWA'); from wwa import *

ERROR 1: PROJ: proj_create_from_database: Open of /home/clair/miniconda3/envs/wwa/share/proj failed


In [10]:
# run the results
pd.concat([read_results(fnm) for fnm in glob.glob("res/AFR-44_june_total_*.txt")]).to_csv("AFR-44_june-total_results.csv")
pd.concat([read_results(fnm) for fnm in glob.glob("res/AFR-22_june*.txt")]).to_csv("AFR-22_june-total_results.csv")


pd.concat([read_results(fnm) for fnm in glob.glob("res/AFR-44_rs_onset*.txt")]).to_csv("AFR-44_rs-onset_results.csv")
pd.concat([read_results(fnm) for fnm in glob.glob("res/AFR-22_rs_onset*.txt")]).to_csv("AFR-22_rs-onset_results.csv")


pd.concat([read_results(fnm) for fnm in glob.glob("res/AFR-44_rs_duration*.txt")]).to_csv("AFR-44_rs-duration_results.csv")
pd.concat([read_results(fnm) for fnm in glob.glob("res/AFR-22_rs_duration*.txt")]).to_csv("AFR-22_rs-duration_results.csv")

In [21]:
[results_to_spreadsheet(fnm) for fnm in glob.glob("AFR-*_results.csv")]

[None, None, None, None, None, None]

In [20]:
def results_to_spreadsheet(fnm):
    
    # load compiled model output
    df = pd.read_csv(fnm, index_col = 0)
    
    res = []
    # loop over each dataset in turn, extracting necessary information from each layer
    for d in sorted(set(df.dataset.values)):
        
        # extract relevant data
        dfd = df.loc[df.dataset == d]
        
        if len(dfd) < 3: 
            print("Missing evaluation")
            print(dfd.index.values)
            continue
        
        if not all([len(set(dfd[attr].values)) == 1 for attr in ["distribution", "fit_type", "covariate_matched", "lower_tail", "include_event"]]):
            print("Mismatch in fitted model")
            print(dfd[["covariate_matched", "distribution", "fit_type", "lower_tail", "include_event"]])  
            continue
            
        # split into validation/attribution to date/projection information
        val, attr, fut = [dfd.loc[dfd.attr_type == atype].set_index("dataset") if atype in dfd.attr_type.values else None for atype in ["validation", "present", "future"]]
        fut["gmst"] = 2
        
        # select just those elements needed from each section
        # will need to add checks that they actually exist
        val = val[["distribution", "fit_type", "lower_tail", 'sigma_est', 'sigma_lower', 'sigma_upper', 'disp_est', 'disp_lower', 'disp_upper']]
        attr = attr[["return_time", "event_magnitude", "gmst_now", 'pr_est', 'pr_lower', 'pr_upper', 'DI_est', 'DI_lower', 'DI_upper']]
        fut = fut[["gmst",'pr_est', 'pr_lower', 'pr_upper', 'DI_est', 'DI_lower', 'DI_upper']]
        
        res.append(val.merge(attr, left_index = True, right_index = True).merge(fut, left_index = True, right_index = True))
    
    pd.concat(res).to_csv(re.sub("results", "spreadsheet", fnm))

In [116]:
# all time series to .csv for upload to Google Drive

def xr_to_pd(da):
    
    return pd.concat([da.sel(run = r, drop = True).to_dataframe(name = r).transpose() for r in da.run.values]).transpose()
    
xr_to_pd(xr.open_dataset("ts/june_totals-afr44.nc").june_total).to_csv("jun-totals_cordex-afr44.csv")
xr_to_pd(xr.open_dataset("ts/june_totals-afr22.nc").june_total).to_csv("jun-totals_cordex-afr22.csv")
xr_to_pd(xr.open_dataset("ts/june_totals-highresmip.nc").june_total).to_csv("jun-totals_highresmip.csv")

xr_to_pd(xr.open_dataset("ts/season_onsets-afr44.nc").rs_onset).to_csv("rs-onset_cordex-afr44.csv")
xr_to_pd(xr.open_dataset("ts/season_onsets-afr22.nc").rs_onset).to_csv("rs-onset_cordex-afr22.csv")
xr_to_pd(xr.open_dataset("ts/season_onsets-highresmip.nc").rs_onset).to_csv("rs-onset_highresmip.csv")

xr_to_pd(xr.open_dataset("ts/season_durations-afr44.nc").rs_duration).to_csv("rs-duration_cordex-afr44.csv")
xr_to_pd(xr.open_dataset("ts/season_durations-afr22.nc").rs_duration).to_csv("rs-duration_cordex-afr22.csv")
xr_to_pd(xr.open_dataset("ts/season_durations-highresmip.nc").rs_duration).to_csv("rs-duration_highresmip.csv")
